# ScmRun

*Suggestions for update:* add examples of handling of timeseries interpolation plus how the guessing works

In this notebook we provide an overview of the capabilities provided by scmdata's `ScmRun` class.
This class is a wrapper around `scmdata.timeseries.TimeSeries` objects, themselves a wrapper around [the `xarray.DataArray` class](https://xarray.pydata.org/en/stable/data-structures.html#dataarray).
The wrappers allow us to make the most of the tools provided by [xarray](https://xarray.pydata.org/en/stable/why-xarray.html) and [pandas](https://pandas.pydata.org/pandas-docs/stable) whilst adding our own requirements on top.

## Imports

In [1]:
# NBVAL_IGNORE_OUTPUT
import traceback

from pint.errors import DimensionalityError

from scmdata import ScmRun

## Loading data

`ScmRun`'s can read many different data types and be loaded in many different ways.
For a full explanation, see the docstring of `ScmRun`'s `__init__` method.

In [2]:
print(ScmRun.__init__.__doc__)


        Initialize the container with timeseries data.

        Parameters
        ----------
        data: Union[ScmDataFrame, ScmRun, IamDataFrame, pd.DataFrame, np.ndarray, str]
            If a :class`ScmDataFrame` or :class`ScmRun` object is provided, then a new
            :obj`ScmRun` is created with a copy of the values and metadata from :obj`data`.

            A :class`pd.DataFrame with IAMC-format data columns (the result
            from :func`ScmRun.timeseries()`) can be provided without any additional
            :obj:`columns` and :obj:`index` information.

            If a numpy array of timeseries data is provided, :obj:`columns` and :obj:`index`
            must also be specified.
            The shape of the numpy array should be ```(n_times, n_series)``` where `n_times`
             is the number of timesteps and `n_series` is the number of time series.

            If a string is passed, data will be attempted to be read from file. Currently,
            reading f

Here we load data from a file.

*Note:* here we load RCP26 emissions data. This originally came from http://www.pik-potsdam.de/~mmalte/rcps/ and has since been re-written into a format which can be read by scmdata using the [pymagicc](https://github.com/openclimatedata/pymagicc) library. We are not currently planning on importing Pymagicc's readers into scmdata by default, please raise an issue [here](https://github.com/openscm/scmdata/issues) if you would like us to consider doing so.

In [3]:
rcp26 = ScmRun("rcp26_emissions.csv", lowercase_cols=True)

## Timeseries

`ScmDataFrame` is ideally suited to working with timeseries data.
The `timeseries` method allows you to easily get the data back in wide format as a *pandas* `DataFrame`.
Here 'wide' format refers to representing timeseries as a row with metadata being contained in the row labels.

In [4]:
rcp26.timeseries().head()

time                                                 2000-01-01 00:00:00  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                4.375663   
                      Emissions|C2F6  kt C2F6 / yr              0.050576   
                      Emissions|C6F14 kt C6F14 / yr             0.000000   
                      Emissions|CCl4  kt CCl4 / yr              8.473000   
                      Emissions|CF4   kt CF4 / yr               0.010377   

time                                                 2000-01-01 00:00:06  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  7.8048   
                      Emissions|C2F6  kt C2F6 / yr                2.3749   
                      Emissions|C6F14 kt C6F14 / yr               0.4624   
                      Emissions|CCl4  kt CCl4 / yr               74.1320   
                      Emissions|CF4   kt CF4 / yr                12.0001   

time                                                 2000-01-01 00:00:12  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  3.3578   
                      Emissions|C2F6  kt C2F6 / yr                0.0857   
                      Emissions|C6F14 kt C6F14 / yr               0.0887   
                      Emissions|CCl4  kt CCl4 / yr                0.0000   
                      Emissions|CF4   kt CF4 / yr                 1.0920   

time                                                 2000-01-01 00:00:18  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  3.3578   
                      Emissions|C2F6  kt C2F6 / yr                0.0857   
                      Emissions|C6F14 kt C6F14 / yr               0.0887   
                      Emissions|CCl4  kt CCl4 / yr                0.0000   
                      Emissions|CF4   kt CF4 / yr                 1.0920   

time                                                 2000-01-01 00:00:24  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  3.3578   
                      Emissions|C2F6  kt C2F6 / yr                0.0857   
                      Emissions|C6F14 kt C6F14 / yr               0.0887   
                      Emissions|CCl4  kt CCl4 / yr                0.0000   
                      Emissions|CF4   kt CF4 / yr                 1.0920   

time                                                 2000-01-01 00:00:30  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  3.3578   
                      Emissions|C2F6  kt C2F6 / yr                0.0857   
                      Emissions|C6F14 kt C6F14 / yr               0.0887   
                      Emissions|CCl4  kt CCl4 / yr                0.0000   
                      Emissions|CF4   kt CF4 / yr                 1.0920   

time                                                 2000-01-01 00:00:36  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                  3.3578   
                      Emissions|C2F6  kt C2F6 / yr                0.0857   
                      Emissions|C6F14 kt C6F14 / yr               0.0887   
                      Emissions|CCl4  kt CCl4 / yr                0.0000   
                      Emissions|CF4   kt CF4 / yr                 1.0920   

time                                                 2001-01-01 00:00:00  \
model scenario region variable        unit                                 
IMAGE RCP26    World  Emissions|BC    Mt BC / yr                4.417385   
                      Emissions|C2F6  kt 

In [5]:
type(rcp26.timeseries())

pandas.core.frame.DataFrame

## Unit conversion

The scmdata package uses the [OpenSCM-Units](https://openscm-units.readthedocs.io/) unit registry and uses the [Pint](https://github.com/hgrecco/pint) library to handle unit conversion.

Calling the `convert_unit` method of an `ScmRun` returns a new `ScmRun` instance with converted units.

In [6]:
rcp26.filter(variable="Emissions|BC").timeseries()

,,,,time,2000-01-01 00:00:00,2000-01-01 00:00:06,2000-01-01 00:00:12,2000-01-01 00:00:18,2000-01-01 00:00:24,2000-01-01 00:00:30,2000-01-01 00:00:36,2001-01-01 00:00:00,2001-01-01 00:00:06,2001-01-01 00:00:12,...,1890-01-01 00:00:00,1891-01-01 00:00:00,1892-01-01 00:00:00,1893-01-01 00:00:00,1894-01-01 00:00:00,1895-01-01 00:00:00,1896-01-01 00:00:00,1897-01-01 00:00:00,1898-01-01 00:00:00,1899-01-01 00:00:00
model,scenario,region,variable,unit,,,,,,,,,,,,,,,,,,,,,
IMAGE,RCP26,World,Emissions|BC,Mt BC / yr,4.375663,7.8048,3.3578,3.3578,3.3578,3.3578,3.3578,4.417385,7.8945,3.3578,...,4.056354,4.088285,4.120215,4.152146,4.184077,4.216008,4.247939,4.27987,4.311801,4.343732


In [7]:
rcp26.filter(variable="Emissions|BC").convert_unit("kg BC / day").timeseries()

,,,,time,1765-01-01 00:00:00,1766-01-01 00:00:00,1767-01-01 00:00:00,1768-01-01 00:00:00,1769-01-01 00:00:00,1770-01-01 00:00:00,1771-01-01 00:00:00,1772-01-01 00:00:00,1773-01-01 00:00:00,1774-01-01 00:00:00,...,2068-01-01 00:00:12,2068-01-01 00:00:18,2068-01-01 00:00:24,2068-01-01 00:00:30,2069-01-01 00:00:00,2069-01-01 00:00:06,2069-01-01 00:00:12,2069-01-01 00:00:18,2069-01-01 00:00:24,2069-01-01 00:00:30
model,scenario,region,variable,unit,,,,,,,,,,,,,,,,,,,,,
IMAGE,RCP26,World,Emissions|BC,kg BC / day,0.0,292944.558522,365181.6564,437636.605065,510316.112252,583227.186858,656376.947296,729772.785763,803422.313484,877333.360712,...,9.193155e+06,9.193155e+06,9.193155e+06,9.193155e+06,1.512158e+07,1.092862e+07,9.193155e+06,9.193155e+06,9.193155e+06,9.193155e+06


Note that you must filter your data first as the unit conversion is applied to all available variables. If you do not, you will receive `DimensionalityError`'s.

In [8]:
try:
    rcp26.convert_unit("kg BC / day").timeseries()
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
pint.errors.DimensionalityError: Cannot convert from 'C * gigametric_ton / a' ([carbon] * [mass] / [time]) to 'BC * kilogram / day' ([black_carbon] * [mass] / [time])


Having said this, thanks to Pint's idea of contexts, we are able to trivially convert to CO<sub>2</sub> equivalent units (as long as we restrict our conversion to variables which have a CO<sub>2</sub> equivalent).

In [9]:
rcp26.filter(variable=["*CO2*", "*CH4*", "*N2O*"]).timeseries()

time                                                                           2000-01-01 00:00:00  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr            123.739050   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                0.653206   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                0.534000   
                      Emissions|N2O                              Mt N2ON / yr             0.932522   

time                                                                           2000-01-01 00:00:06  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              300.2069   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                  1.1488   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                  6.7350   
                      Emissions|N2O                              Mt N2ON / yr               7.4566   

time                                                                           2000-01-01 00:00:12  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              142.0527   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                  0.5113   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                 -0.9308   
                      Emissions|N2O                              Mt N2ON / yr               5.2823   

time                                                                           2000-01-01 00:00:18  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              142.0527   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                  0.0000   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                 -0.9308   
                      Emissions|N2O                              Mt N2ON / yr               5.2823   

time                                                                           2000-01-01 00:00:24  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              142.0527   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                  0.0000   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                 -0.9308   
                      Emissions|N2O                              Mt N2ON / yr               5.2823   

time                                                                           2000-01-01 00:00:30  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              142.0527   
                      Emissions|CO2|MAGICC AFOLU                 Gt C / yr                  0.0000   
                      Emissions|CO2|MAGICC Fossil and Industrial Gt C / yr                 -0.9308   
                      Emissions|N2O                              Mt N2ON / yr               5.2823   

time                                                                           2000-01-01 00:00:36  \
model scenario region variable                                   unit                                
IMAGE RCP26    World  Emissions|CH4                              Mt CH4 / yr              142.0527   
                

In [10]:
rcp26.filter(variable=["*CO2*", "*CH4*", "*N2O*"]).convert_unit(
    "Mt CO2 / yr", context="AR4GWP100"
).timeseries()

time                                                                                       1765-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100                     0.0   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100                     0.0   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100                    11.0   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                     0.0   

time                                                                                       1766-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               49.081547   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               19.573752   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                2.430911   

time                                                                                       1767-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               60.911202   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               39.147508   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                4.737559   

time                                                                                       1768-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               72.777625   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               58.721260   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                7.044330   

time                                                                                       1769-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               84.681955   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               78.295012   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                9.351227   

time                                                                                       1770-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               96.625365   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               97.868767   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
     

Without the context, a `DimensionalityError` is once again raised.

In [11]:
try:
    rcp26.convert_unit("Mt CO2 / yr").timeseries()
except DimensionalityError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
pint.errors.DimensionalityError: Cannot convert from 'BC * megametric_ton / a' ([black_carbon] * [mass] / [time]) to 'CO2 * megametric_ton / a' ([carbon] * [mass] / [time])


In addition, when we do a conversion with contexts, the context information is automatically added to the metadata. This ensures we can't accidentally use a different context for further conversions.

In [12]:
ar4gwp100_converted = rcp26.filter(variable=["*CO2*", "*CH4*", "*N2O*"]).convert_unit(
    "Mt CO2 / yr", context="AR4GWP100"
)
ar4gwp100_converted.timeseries()

time                                                                                       1765-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100                     0.0   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100                     0.0   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100                    11.0   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                     0.0   

time                                                                                       1766-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               49.081547   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               19.573752   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                2.430911   

time                                                                                       1767-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               60.911202   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               39.147508   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                4.737559   

time                                                                                       1768-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               72.777625   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               58.721260   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                7.044330   

time                                                                                       1769-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               84.681955   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               78.295012   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
                      Emissions|N2O                              Mt CO2 / yr AR4GWP100                9.351227   

time                                                                                       1770-01-01 00:00:00  \
model scenario region variable                                   unit        unit_context                        
IMAGE RCP26    World  Emissions|CH4                              Mt CO2 / yr AR4GWP100               96.625365   
                      Emissions|CO2|MAGICC AFOLU                 Mt CO2 / yr AR4GWP100               97.868767   
                      Emissions|CO2|MAGICC Fossil and Industrial Mt CO2 / yr AR4GWP100               11.000000   
     

Trying to convert without a context, or with a different context, raises an error.

In [13]:
try:
    ar4gwp100_converted.convert_unit("Mt CO2 / yr")
except ValueError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
ValueError: Existing unit conversion context(s), `['AR4GWP100']`, doesn't match input context, `None`, drop `unit_context` metadata before doing conversion


In [14]:
try:
    ar4gwp100_converted.convert_unit("Mt CO2 / yr", context="AR5GWP100")
except ValueError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
ValueError: Existing unit conversion context(s), `['AR4GWP100']`, doesn't match input context, `AR5GWP100`, drop `unit_context` metadata before doing conversion


## Convenience methods

Below we showcase a few convenience methods of `ScmRun`. These will grow over time, please add a pull request adding more where they are useful!

### get_unique_meta

This method helps with getting the unique metadata values in an `ScmRun`. Here we show how it can be useful. Check out its docstring for full details. 

By itself, it doesn't do anything special, just returns the unique metadata values as a list.

In [15]:
rcp26.get_unique_meta("variable")

['Emissions|BC',
 'Emissions|C2F6',
 'Emissions|C6F14',
 'Emissions|CCl4',
 'Emissions|CF4',
 'Emissions|CFC11',
 'Emissions|CFC113',
 'Emissions|CFC114',
 'Emissions|CFC115',
 'Emissions|CFC12',
 'Emissions|CH3Br',
 'Emissions|CH3CCl3',
 'Emissions|CH3Cl',
 'Emissions|CH4',
 'Emissions|CO',
 'Emissions|CO2|MAGICC AFOLU',
 'Emissions|CO2|MAGICC Fossil and Industrial',
 'Emissions|HCFC141b',
 'Emissions|HCFC142b',
 'Emissions|HCFC22',
 'Emissions|HFC125',
 'Emissions|HFC134a',
 'Emissions|HFC143a',
 'Emissions|HFC227ea',
 'Emissions|HFC23',
 'Emissions|HFC245fa',
 'Emissions|HFC32',
 'Emissions|HFC4310',
 'Emissions|Halon1202',
 'Emissions|Halon1211',
 'Emissions|Halon1301',
 'Emissions|Halon2402',
 'Emissions|N2O',
 'Emissions|NH3',
 'Emissions|NMVOC',
 'Emissions|NOx',
 'Emissions|OC',
 'Emissions|SF6',
 'Emissions|SOx']

However, it can be useful if you expect there to only be one unique metadata value. In such a case, you can use the `no_duplicates` argument to ensure that you only get a single value as its native type (not a list) and that an error will be raised if this isn't the case.

In [16]:
rcp26.get_unique_meta("model", no_duplicates=True)

'IMAGE'

In [17]:
try:
    rcp26.get_unique_meta("unit", no_duplicates=True)
except ValueError:
    traceback.print_exc(limit=0, chain=False)

Traceback (most recent call last):
ValueError: `unit` column is not unique (found values: ['Mt BC / yr', 'kt C2F6 / yr', 'kt C6F14 / yr', 'kt CCl4 / yr', 'kt CF4 / yr', 'kt CFC11 / yr', 'kt CFC113 / yr', 'kt CFC114 / yr', 'kt CFC115 / yr', 'kt CFC12 / yr', 'kt CH3Br / yr', 'kt CH3CCl3 / yr', 'kt CH3Cl / yr', 'Mt CH4 / yr', 'Mt CO / yr', 'Gt C / yr', 'kt HCFC141b / yr', 'kt HCFC142b / yr', 'kt HCFC22 / yr', 'kt HFC125 / yr', 'kt HFC134a / yr', 'kt HFC143a / yr', 'kt HFC227ea / yr', 'kt HFC23 / yr', 'kt HFC245fa / yr', 'kt HFC32 / yr', 'kt HFC4310 / yr', 'kt Halon1202 / yr', 'kt Halon1211 / yr', 'kt Halon1301 / yr', 'kt Halon2402 / yr', 'Mt N2ON / yr', 'Mt N / yr', 'Mt NMVOC / yr', 'Mt OC / yr', 'kt SF6 / yr', 'Mt S / yr'])
